<a href="https://colab.research.google.com/github/rociavl/UNI/blob/main/P4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np

# Données d'entrée
ubi_and_pesos = {
    'A': [20, 20, 5],
    'B': [10, 10, 2],
    'C': [40, 10, 4],
    'D': [60, 30, 1]
}
X = 8  # Coût en euros

# Fonction pour calculer l'optimum en une seule dimension (X ou Y)
def rectangular(dim, ubi_pesos, pesos):
    # Agréger les poids pour les localisations identiques
    ubi_pesos = np.array(ubi_pesos)
    pesos = np.array(pesos)

    # Obtenir les localisations uniques et la somme des poids pour chaque localisation
    ubi_pesos_uniques, indices = np.unique(ubi_pesos, return_inverse=True)
    pesos_uniques = np.zeros_like(ubi_pesos_uniques, dtype=float)

    # Additionner les poids pour les localisations identiques
    for i in range(len(pesos)):
        pesos_uniques[indices[i]] += pesos[i]
    peso_total = np.sum(pesos_uniques)
    acumulada = [-peso_total]  # Liste pour stocker l'accumulation des poids
    ubi = []

    # Calcul de l'accumulation des poids
    for i in range(len(pesos_uniques)):
        acu = acumulada[i] + pesos_uniques[i] * 2
        acumulada.append(acu)

    for j in range(len(acumulada) - 1):
        if acumulada[j] * acumulada[j + 1] < 0:
            ubi.append(ubi_pesos_uniques[j])
        elif acumulada[j] == 0:
            ubi.append(ubi_pesos_uniques[j-1])
            ubi.append(ubi_pesos_uniques[j])
    return ubi

# Fonction pour calculer l'optimum en X et Y
def rectangular_x_y(ubi_and_pesos):
    ubi_pesos_x = []
    ubi_pesos_y = []
    pesos = []

    for key, value in ubi_and_pesos.items():
        ubi_pesos_x.append(value[0])  # X
        ubi_pesos_y.append(value[1])  # Y
        pesos.append(value[2])  # Poids

    # Calculer la position optimale pour X et Y
    ubicacion_optima_x = rectangular('X', ubi_pesos_x, pesos)
    ubicacion_optima_y = rectangular('Y', ubi_pesos_y, pesos)

    return ubicacion_optima_x, ubicacion_optima_y

ubicacion_optima_x, ubicacion_optima_y = rectangular_x_y(ubi_and_pesos)

print(f"1. Emplacement optimal en X et Y: {ubicacion_optima_x, ubicacion_optima_y}")

# Fonction pour calculer la moyenne pondérée (méthode quadratique)
def cuadratica(ubi_and_pesos):
    # Extraire les coordonnées X, Y et les poids
    ubi_pesos_x = np.array([value[0] for value in ubi_and_pesos.values()])  # Coordonnées X
    ubi_pesos_y = np.array([value[1] for value in ubi_and_pesos.values()])  # Coordonnées Y
    pesos = np.array([value[2] for value in ubi_and_pesos.values()])  # Poids

    # Calcul de la moyenne pondérée pour X et Y
    moyenne_ponderee_x = np.sum(ubi_pesos_x * pesos) / np.sum(pesos)
    moyenne_ponderee_y = np.sum(ubi_pesos_y * pesos) / np.sum(pesos)

    return moyenne_ponderee_x, moyenne_ponderee_y

moyenne_ponderee_x, moyenne_ponderee_y = cuadratica(ubi_and_pesos)

print(f"2. Méthode quadratique X et Y: {moyenne_ponderee_x, moyenne_ponderee_y}")

# Stocker les emplacements optimaux
ubi_optimas = {
    'rectangular': [ubicacion_optima_x, ubicacion_optima_y],
    'cuadratica': [moyenne_ponderee_x, moyenne_ponderee_y]
}

# Fonction pour calculer le coût rectangulaire (basé sur la somme des valeurs absolues)
def coste_rectangular(ubi_op, ubi_and_pesos,cost_unitari):
    ubi_pesos_x = np.array([value[0] for value in ubi_and_pesos.values()])  # Coordonnées X
    ubi_pesos_y = np.array([value[1] for value in ubi_and_pesos.values()])  # Coordonnées Y
    pesos = np.array([value[2] for value in ubi_and_pesos.values()])  # Poids

    # Coût pour X basé sur l'approche rectangulaire (valeur absolue)
    coste_x_rect = np.sum(np.abs(ubi_pesos_x - ubi_op['rectangular'][0][0]) * pesos)

    # Coût pour Y
    coste_y_rect = np.sum(np.abs(ubi_pesos_y - ubi_op['rectangular'][1][0]) * pesos)

    # Retourner la somme des coûts X et Y
    return (coste_x_rect + coste_y_rect)*cost_unitari, coste_x_rect ,coste_y_rect

# Fonction pour calculer le coût quadratique (basé sur l'écart au carré)
def coste_cuadratica(ubi_op, ubi_and_pesos, cost_unitari):
    ubi_pesos_x = np.array([value[0] for value in ubi_and_pesos.values()])  # Coordonnées X
    ubi_pesos_y = np.array([value[1] for value in ubi_and_pesos.values()])  # Coordonnées Y
    pesos = np.array([value[2] for value in ubi_and_pesos.values()])  # Poids

    # Coût pour X basé sur l'approche quadratique (carré des écarts)
    coste_x_cua  = np.sum((ubi_pesos_x - ubi_op['cuadratica'][0])**2 * pesos)

    # Coût pour Y
    coste_y_cua  = np.sum((ubi_pesos_y - ubi_op['cuadratica'][1])**2 * pesos)

    # Retourner la somme des coûts X et Y
    return (coste_x_cua + coste_y_cua)*cost_unitari, coste_x_cua ,coste_y_cua

# Calcul des coûts
coste_rect, coste_x_rect ,coste_y_rect = coste_rectangular(ubi_optimas, ubi_and_pesos,X)
coste_cuad,coste_x_cua ,coste_y_cua  = coste_cuadratica(ubi_optimas, ubi_and_pesos,X)

# Affichage des résultats
print(f"3. \033[1mCoût rectangulaire: {coste_rect}\033[0m")
print(f"    Coût X: {coste_x_rect}")
print(f"    Coût Y: {coste_y_rect}")
print(f"   Coût quadratique: {coste_cuad}")
print(f"    Coût X: {coste_x_cua}")
print(f"    Coût Y: {coste_y_cua}")


1. Emplacement optimal en X et Y: ([20], [10, 20])
2. Méthode quadratique X et Y: (28.333333333333332, 15.833333333333334)
3. Coût rectangulaire: 1680
    Coût X: 140
    Coût Y: 70
   Coût quadratique: 24466.66666666667
    Coût X: 2566.666666666667
    Coût Y: 491.66666666666674
